In [4]:
import sympy as sp
import sym_functions
from typing import Dict, Tuple, List

# Symbols
t = sp.symbols("t", positive=True, real=True)
T = sp.symbols("T", positive=True, real=True, nonzero=True)
A = sp.symbols("A", positive=True, real=True, nonzero=True)
N = sp.symbols("N", positive=True, real=True, nonzero=True)
sigma_m = sp.symbols("sigma_m", positive=True, real=True, nonzero=True)
omega = sp.symbols("omega", real=True, positive=True, nonzero=True)
phi = sp.symbols("phi", real=True)

# Model
f = A * sp.sin(omega * t + phi)


# Parameter list
params = [A, omega, phi]
param_indices = {param: i for i, param in enumerate(params)}

In [11]:
def simplify_trig_identities(
    expr: sp.Expr, phi: sp.Symbol, print_replacements: bool = True
) -> Tuple[sp.Expr, List[sp.Expr]]:
    """
    Simplifies trig identities with explicit logging via log_and_replace().
    """
    replacements_log: List[sp.Expr] = []

    a = sp.Wild("a", exclude=[phi, sp.sin(phi), sp.cos(phi)])

    def log_and_replace(matched_expr: sp.Expr, replacement: sp.Expr):
        replacements_log.append((matched_expr, replacement))
        return replacement

    # Pattern: a * sin^2(phi) * cos^2(phi) => a * (1 - cos(4phi))/8
    pattern = a * sp.sin(phi) ** 2 * sp.cos(phi) ** 2
    expr = expr.replace(
        pattern, lambda a: log_and_replace(pattern, a * (1 - sp.cos(4 * phi)) / 8)
    )

    # Pattern: a * sin(phi) * cos(phi) => a * sin(2phi)/2
    pattern = a * sp.sin(phi) * sp.cos(phi)
    expr = expr.replace(
        pattern, lambda a: log_and_replace(pattern, a * sp.sin(2 * phi) / 2)
    )

    if print_replacements and replacements_log:
        print("Trig identity simplifications applied to:")
        for old, new in replacements_log:
            print("  ", old, "→", new)

    return expr.simplify()

In [12]:
print("test")
ii = 2
for jj in range(3):
    print(f"FIM[{ii}, {jj}]  [{str(params[ii])}, {str(params[jj])}]  :")
    df1 = sp.diff(f, params[ii])
    df2 = sp.diff(f, params[jj])
    print(f"df1:   {df1}")
    print(f"df2:    {df2}")
    product = sp.expand(df1 * df2)
    print(f"product:    {product}")
    product = simplify_trig_identities(product, phi)
    product = sp.expand(product)
    # product=product.series(t, 0, 2).removeO().expand()
    print(product)
    print("\n")

test
FIM[2, 0]  [phi, A]  :
df1:   A*cos(omega*t + phi)
df2:    sin(omega*t + phi)
product:    A*sin(omega*t + phi)*cos(omega*t + phi)
A*sin(2*omega*t + 2*phi)/2


FIM[2, 1]  [phi, omega]  :
df1:   A*cos(omega*t + phi)
df2:    A*t*cos(omega*t + phi)
product:    A**2*t*cos(omega*t + phi)**2
A**2*t*cos(omega*t + phi)**2


FIM[2, 2]  [phi, phi]  :
df1:   A*cos(omega*t + phi)
df2:    A*cos(omega*t + phi)
product:    A**2*cos(omega*t + phi)**2
A**2*cos(omega*t + phi)**2




In [13]:
from sympy import Matrix, symbols, integrate, diff, simplify, eye
from typing import Dict, Tuple
from multiprocessing import Pool, cpu_count

from typing import Tuple, List


# Update the integral computation function to include simplification inside the parallelized task
def compute_integral_pair_simplified(
    args: Tuple[sp.Expr, sp.Expr, sp.Symbol, sp.Symbol, sp.Symbol, sp.Symbol],
) -> sp.Expr:
    df1, df2, t, T, R, sigma_m, phi = args
    product = sp.expand(df1 * df2)
    product = sp.simplify(product)
    product = simplify_trig_identities(product, omega, phi)
    integral = sp.integrate(product, (t, 0, T))
    scaled = integral * R / sigma_m**2
    expr = sp.expand(scaled)  # Expand basic algebra first
    expr = sp.expand_trig(expr)  # Expand trig products (e.g., sin^2)
    expr = sp.factor(expr)  # Pull out common exponentials
    expr = sp.cancel(expr)  # Clean up rational forms
    expr = sp.collect(expr, T)  # Group by powers of T (or lambda_D, etc.)
    # expr = sp.simplify(expr)
    return expr


def compute_fisher_matrix_serial_simplified(
    model: sp.Expr,
    parameters: list[sp.Symbol],
    t: sp.Symbol,
    T: sp.Symbol,
    sigma_m: sp.Symbol,
    N: sp.Symbol,
) -> sp.Matrix:
    """
    Computes the Fisher Information Matrix (FIM) without parallelism,
    using simplified symbolic integrals over all parameter pairs.

    Args:
        model: symbolic expression of the model function
        parameters: list of parameters to differentiate with respect to
        t: time symbol
        T: total acquisition time symbol
        sigma_m: measurement noise
        N: number of samples

    Returns:
        sympy.Matrix: symbolic Fisher Information Matrix
    """
    R = N / T
    n = len(parameters)

    entries: list[sp.Expr] = []

    for p1 in parameters:
        df1 = sp.diff(model, p1)
        for p2 in parameters:
            df2 = sp.diff(model, p2)
            result = compute_integral_pair_simplified((df1, df2, t, T, R, sigma_m, phi))
            entries.append(result)

    return sp.Matrix(n, n, entries)

In [14]:
# runtime ~2 s
FIM = compute_fisher_matrix_serial_simplified(f, params, t, T, sigma_m, N)

In [15]:
FIM

Matrix([
[(N*sin(phi)**2*sin(T*omega)*cos(T*omega) + N*sin(phi)*sin(T*omega)**2*cos(phi) - N*sin(phi)*cos(phi)*cos(T*omega)**2 + N*sin(phi)*cos(phi) - N*sin(T*omega)*cos(phi)**2*cos(T*omega) + T*(N*omega*sin(phi)**2*sin(T*omega)**2 + N*omega*sin(phi)**2*cos(T*omega)**2 + N*omega*sin(T*omega)**2*cos(phi)**2 + N*omega*cos(phi)**2*cos(T*omega)**2))/(2*T*omega*sigma_m**2),                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  (2*A*N*sin(phi)*cos(phi)*cos(T*omega)**2 - 2*A*N*si

In [16]:
# runtime ~ 30s
FIM_inv = sym_functions.inverse_via_symmetric_substitution(FIM)

creating template for symbolic symmetric inverse
applying substitution to symbolic inverse


In [ ]:
# Param list and symbolic variables

param_vars = {}

# T omega = R
R = sp.symbols("R", real=True, positive=True, nonzero=True)

expansion_pt = sp.Rational(1, 2) * 1 / (2 * sp.pi)


# Rewrite and simplify each diagonal entry
param = params[0]
idx = param_indices[param]
expr = FIM_inv[idx, idx]
expr_sub = expr.subs(omega, R / T)
expr_sub = sp.collect(expr_sub, R)
expr_sub = sp.simplify(expr_sub)
expr_series = expr_sub.series(R, expansion_pt, 5).removeO().expand()
expr_series = sp.simplfy(expr_series)
sp.simplify(expr_series)

In [34]:
expr.subs(omega, R / T)

(T**2*(A**2*N*sin(R)**2*sin(phi)**2 - 2*A**2*N*sin(R)*sin(phi)*cos(R)*cos(phi) + A**2*N*cos(R)**2*cos(phi)**2 - A**2*N*cos(phi)**2 + T**2*(A**2*N*R**2*sin(R)**2*sin(phi)**2/T**2 + A**2*N*R**2*sin(R)**2*cos(phi)**2/T**2 + A**2*N*R**2*sin(phi)**2*cos(R)**2/T**2 + A**2*N*R**2*cos(R)**2*cos(phi)**2/T**2) + T*(-2*A**2*N*R*sin(R)**2*sin(phi)*cos(phi)/T - 2*A**2*N*R*sin(R)*sin(phi)**2*cos(R)/T + 2*A**2*N*R*sin(R)*cos(R)*cos(phi)**2/T + 2*A**2*N*R*sin(phi)*cos(R)**2*cos(phi)/T))**2/(16*R**4*sigma_m**4) - T**2*(-A**2*N*sin(R)**2*sin(phi)*cos(phi) - A**2*N*sin(R)*sin(phi)**2*cos(R) + A**2*N*sin(R)*cos(R)*cos(phi)**2 + A**2*N*sin(phi)*cos(R)**2*cos(phi) - A**2*N*sin(phi)*cos(phi) + T*(A**2*N*R*sin(R)**2*sin(phi)**2/T + A**2*N*R*sin(R)**2*cos(phi)**2/T + A**2*N*R*sin(phi)**2*cos(R)**2/T + A**2*N*R*cos(R)**2*cos(phi)**2/T))*(3*A**2*N*sin(R)**2*sin(phi)*cos(phi) + 3*A**2*N*sin(R)*sin(phi)**2*cos(R) - 3*A**2*N*sin(R)*cos(R)*cos(phi)**2 - 3*A**2*N*sin(phi)*cos(R)**2*cos(phi) + 3*A**2*N*sin(phi)*cos(ph

In [ ]:
expr_series

In [ ]:
import sympy as sp

# Param list and symbolic variables
param_vars = {}

# Rewrite and simplify each diagonal entry
for param in params:
    idx = param_indices[param]
    var = FIM_inv[idx, idx]

    var_hyper = var.rewrite(sp.cosh)
    var_simplified = sp.simplify(var_hyper)

    param_vars[param] = var_simplified

KeyboardInterrupt: 